In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,DataCollatorWithPadding
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.amp import autocast
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


#### The data used in this jupyter note nookbook can be found there :https://lre.baulab.info/data/commonsense/

In [2]:
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cup')
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return the number of available GPUs
print(torch.cuda.get_device_name(0))  # Should return the name of the GPU

True
1
NVIDIA GeForce GTX 1080 Ti


In [3]:
os.listdir("/home/students/wli/Mechanistic_Interpretability/Extract_Attricute/Qwen")

['Qwen2.5-1.5B']

In [4]:

model_dict = ["Qwen/Qwen2.5-0.5B","Qwen/Qwen2.5-1.5B","Qwen/Qwen2.5-3B"]
model_num = 1
local_dir = f"/home/students/wli/Mechanistic_Interpretability/Extract_Attricute/{model_dict[model_num]}"

if os.path.exists(local_dir):
    tokenizer = AutoTokenizer.from_pretrained(local_dir,padding_side = 'left')
    model = AutoModelForCausalLM.from_pretrained(local_dir)

else: 
    os.mkdir(local_dir)
    tokenizer = AutoTokenizer.from_pretrained(model_dict[model_num],padding_side = 'left',cache_dir = local_dir)
    model = AutoModelForCausalLM.from_pretrained(model_dict[model_num],cache_dir = local_dir).to(device)
    tokenizer.save_pretrained(local_dir)
    model.save_pretrained(local_dir)
model.eval()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [5]:
print('hellow world')

hellow world


In [6]:
import json

with open('Extract_Attricute/output.json', 'r') as f:
    data = json.load(f)

In [7]:
data

{'name': 'fruit outside color',
 'prompt_templates': ['On the outside, {} are'],
 'prompt_templates_zs': ['What color are {} on the outside? They are the color of'],
 'properties': {'relation_type': 'commonsense',
  'domain_name': 'fruit',
  'range_name': 'color',
  'symmetric': False},
 'samples': [{'subject': 'bananas', 'object': 'yellow'},
  {'subject': 'apples', 'object': 'red'},
  {'subject': 'watermelons', 'object': 'green'},
  {'subject': 'kiwis', 'object': 'brown'},
  {'subject': 'dragon fruits', 'object': 'pink'},
  {'subject': 'eggplants', 'object': 'purple'},
  {'subject': 'zucchinis', 'object': 'green'},
  {'subject': 'pineapples', 'object': 'brown'},
  {'subject': 'mangoes', 'object': 'green'},
  {'subject': 'cucumbers', 'object': 'green'},
  {'subject': 'radishes', 'object': 'pink'},
  {'subject': 'passion fruits', 'object': 'purple'},
  {'subject': 'nectarines', 'object': 'red'},
  {'subject': 'plums', 'object': 'purple'},
  {'subject': 'potatos', 'object': 'brown'},
  {

In [8]:
tokenizer(['this is a test','asd'], return_tensors='pt',max_length= 2,padding= True,truncation=True)

{'input_ids': tensor([[   574,    374],
        [151643,  79102]]), 'attention_mask': tensor([[1, 1],
        [0, 1]])}

In [9]:
# to check if each sample can be encoded by the model within the next 20 generated tokens
def check_data(data, model, tokenizer, batch_size=1):
    model.to(device)
    

    max_new_tokens = 21
    
    generated_ids = []

    filtered_data = []

    prompt_tmp = 'One the outside, the color of '
    prompt_baselines = []
    for i in range(len(data['samples'])):
            prompt_baseline = prompt_tmp + data['samples'][i]['subject']
            prompt_baselines.append(prompt_baseline)

    num_batches = len(prompt_baselines) / batch_size



    if len(prompt_baselines) % batch_size == 0:
        pass
    else:
        num_batches = int(num_batches) + 1


    for i in range(num_batches):
        start_idx = i * num_batches
        end_idx = min(start_idx + batch_size, len(prompt_baselines))

        batched_prompts = prompt_baselines[start_idx:end_idx]

        model_inputs = tokenizer(batched_prompts, return_tensors='pt', padding= True).to(device)
        generated_id_batch = model.generate(**model_inputs,
                max_new_tokens = max_new_tokens
                )
        docoded_outputs= tokenizer.batch_decode(generated_id_batch,skip_special_tokens=True)


        for j in range(len(docoded_outputs)):
             if data['samples'][start_idx + j]['object']  in docoded_outputs[j]:
                  filtered_data.append(data['samples'][start_idx+j])
                  
        
    return filtered_data
             
             



from the result above, we can see that Qwen 2.5-3B is better than GPT-J - 6B

In [10]:
filtered_data = check_data(data,model,tokenizer,16)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [11]:
print(filtered_data)
print(f" the lenght of filtered_data is {len(filtered_data)}")

[{'subject': 'bananas', 'object': 'yellow'}, {'subject': 'apples', 'object': 'red'}, {'subject': 'watermelons', 'object': 'green'}, {'subject': 'eggplants', 'object': 'purple'}, {'subject': 'zucchinis', 'object': 'green'}, {'subject': 'cucumbers', 'object': 'green'}, {'subject': 'passion fruits', 'object': 'purple'}, {'subject': 'nectarines', 'object': 'red'}, {'subject': 'grapefruits', 'object': 'orange'}, {'subject': 'watermelons', 'object': 'green'}, {'subject': 'eggplants', 'object': 'purple'}, {'subject': 'zucchinis', 'object': 'green'}, {'subject': 'cucumbers', 'object': 'green'}, {'subject': 'passion fruits', 'object': 'purple'}, {'subject': 'nectarines', 'object': 'red'}, {'subject': 'grapefruits', 'object': 'orange'}, {'subject': 'oranges', 'object': 'orange'}]
 the lenght of filtered_data is 17


In [12]:
# for each filtered_data

# sample 

import dataset

In [13]:
from datasets import load_dataset

ds = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1",split='train',streaming=True)
ds.shuffle(seed = 50)

IterableDataset({
    features: ['text'],
    num_shards: 2
})

In [14]:
ds_it = iter(ds)

In [15]:
next(ds_it)

{'text': ''}

In [16]:
list(ds.take(5))

[{'text': ''},
 {'text': ' = Valkyria Chronicles III = \n'},
 {'text': ''},
 {'text': ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'},
 {'text': " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent mult

In [17]:
import random
def sample_utterance(sigma, dataset, num_samples = 5):
    
    matching_lines = [line['text'] for line in dataset if sigma in line['text']]
    if len(matching_lines) < num_samples:
        print("not enough utterance")
    else:
        sampled_utterance = random.sample(matching_lines,num_samples)

    
    return sampled_utterance

def truncat_to_window(utterance, sigma,tokenizer, window_size = 20,last_target_flag : bool = False):

    """
    utterance : List[Dict['text': str]]  with_size being 5 by default
    
    in order to patch the hidden representation of the last subject's token to the target model, 
    I choose to set subject to be the last few tokens of sampled_utterance.
    Cause tokenizer use the Byte encoding wordpiece



    return : list[(token_id, offset, sigma_len)]

    """
    
    sigma_tokens = tokenizer.tokenize(sigma)

    sigma_len = len(sigma_tokens)

    prefix_len = 3

    input_ids =[]

    for line in utterance:
        tokens = tokenizer.tokenize(line)

        # find the start index of sigma_tokens
        sigma_start = 0

        for i in range(len(tokens)-sigma_len + 1):

            if tokenizer.convert_tokens_to_string(tokens[i:i + sigma_len]).lower() == tokenizer.convert_tokens_to_string(sigma_tokens).lower():
                sigma_start = i
                break
        
        # print(sigma_start)
        # I tend to keep some prefix before suject as possible so that 
        max_start = max(0,  sigma_start - prefix_len)
        min_start = max(0, sigma_start - window_size + sigma_len)


        start_idx = random.randint(min_start, max_start)
        if last_target_flag:
            end_idx = sigma_start + sigma_len
     
        else:
            end_idx = min(start_idx + window_size, len(line))

        # print(start_idx, end_idx) # for debug reasoning

        truncated_tokens = tokens[start_idx:end_idx]  # [ids]

        input_ids.append((truncated_tokens,(sigma_start - start_idx, sigma_len)))

    return input_ids

In [18]:
# a = sample_utterance(' bananas', ds)

In [19]:
# a

In [20]:
# truncat_to_window(a,' bananas',tokenizer)

In [21]:
# for line in truncat_to_window(a,' bananas',tokenizer):
#     print(tokenizer.convert_tokens_to_string(line[0]))
#     print(line[0])
#     print(line[1])
#     print(tokenizer.convert_tokens_to_ids(line[0]))
#     print(" $$$$$$$$")

' banana' is not in 'banana'
space is very important


In [22]:
filtered_data

[{'subject': 'bananas', 'object': 'yellow'},
 {'subject': 'apples', 'object': 'red'},
 {'subject': 'watermelons', 'object': 'green'},
 {'subject': 'eggplants', 'object': 'purple'},
 {'subject': 'zucchinis', 'object': 'green'},
 {'subject': 'cucumbers', 'object': 'green'},
 {'subject': 'passion fruits', 'object': 'purple'},
 {'subject': 'nectarines', 'object': 'red'},
 {'subject': 'grapefruits', 'object': 'orange'},
 {'subject': 'watermelons', 'object': 'green'},
 {'subject': 'eggplants', 'object': 'purple'},
 {'subject': 'zucchinis', 'object': 'green'},
 {'subject': 'cucumbers', 'object': 'green'},
 {'subject': 'passion fruits', 'object': 'purple'},
 {'subject': 'nectarines', 'object': 'red'},
 {'subject': 'grapefruits', 'object': 'orange'},
 {'subject': 'oranges', 'object': 'orange'}]

In [23]:
from torch.utils.data import DataLoader
from datasets import Dataset

In [24]:
config = model.config
num_layers = config.num_hidden_layers

target_prompt = "On the outside, the color of x"
inputs = []

In [ ]:
for item in filtered_data:
    subj = ' ' +  item['subject']  # becasue of the tokenizer's property, ' ' should be added. To keey consistency 
    obj = item['object']

    sampled_subj = sample_utterance(subj,load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1",split='train',streaming=True))

    processed_sampled_utterance = truncat_to_window(sampled_subj, subj,tokenizer,last_target_flag=True)





    for tokens, sigma_pos in processed_sampled_utterance:
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        
        inputs.append({'input_ids':input_ids,'obj':obj})
        # inputs['input_ids']append(input_ids)
        # inputs['object'].append(obj)
        # inputs['target_pos'].append(sigma_pos)


    break


eval_data = Dataset.from_list(inputs)


In [25]:
# eval_data.save_to_disk('./')

In [26]:
eval_data = Dataset.load_from_disk('./')

In [27]:
eval_data

Dataset({
    features: ['input_ids', 'obj'],
    num_rows: 5
})

In [28]:
tokenizer(' bananas')

{'input_ids': [67342], 'attention_mask': [1]}

In [29]:
collator = DataCollatorWithPadding(tokenizer)
dataloader = DataLoader(
    eval_data.remove_columns(['obj']),
    4,
    collate_fn=collator
)

In [ ]:
batch_size = 4

# first forward pass on source prompts
with torch.no_grad():

    for batch in dataloader:
        
        batch.to(device)

        print(f" the batch size is {len(batch)}")
        
            
        # first forward pass on source prompts
        source_output = model(input_ids = batch['input_ids'],attention_mask = batch['attention_mask'],output_hidden_states = True)
        
        source_hidden_state = source_output.hidden_states[1:]

        
        #second forward pass on target prompts
        for layer in source_hidden_state:

            # print(f"the size of source hidden states is {layer.shape}")

            source_layer_hidden_states = layer[:,-1,:]   #(batch_size, sequence, hidden_dimension)

            for layer_target in range(num_layers):

                # print(f"the size of patched hidden state is {layer_hidden_states.shape}")


                first_forwardpass_flag = True

                def forward_hook(module, input, output):
                    global first_forwardpass_flag
                    if first_forwardpass_flag:

                        print(f"the size of output is ************************************************{output[0].shape}")
                        # print(f"the size of layer hidden states {source_layer_hidden_states.shape}")
            
                        output[0][: , -1 , :] = source_layer_hidden_states

                        first_forwardpass_flag = False
                        
                

                hook_handle = model.model.layers[layer_target].register_forward_hook(forward_hook)
                generated_ids = model.generate(**tokenizer([target_prompt for i in range(len(batch['input_ids']))],padding = True,return_tensors='pt').to(device),max_new_tokens = 20)
                hook_handle.remove()

                decoded_output = tokenizer.batch_decode(generated_ids,skip_special_tokens=True)
                print(decoded_output)

            #     break
            # break

        # break
            

 the batch size is 2


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


the size of output is ************************************************torch.Size([4, 8, 1536])


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


['On the outside, the color of x is yellow. However, when they are cut open, the inside is white. This is because the', 'On the outside, the color of x is yellow. However, when they are cut open, the inside is white. This is because the', 'On the outside, the color of x is yellow. However, when they are cut open, the inside is white. This is because the', 'On the outside, the color of x is yellow. However, when they are cut open, the inside is white. This is because the']
the size of output is ************************************************torch.Size([4, 8, 1536])


...

['On the outside, the color of x is yellow, but on the inside, the color of bananas is yellow. The color of bananas is', 'On the outside, the color of x is yellow, but on the inside, it is white. The inside of the banana is white because', 'On the outside, the color of x is yellow, but on the inside, it is white. The inside of the banana is white because', 'On the outside, the color of x is yellow. On the inside, the color of bananas is yellow. On the outside, the color']
the size of output is ************************************************torch.Size([4, 8, 1536])


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


['On the outside, the color of x is green. On the inside, the color of bananas is yellow. On the outside, the color', 'On the outside, the color of x is yellow, but on the inside, it is white. The inside of the banana is white,', 'On the outside, the color of x is yellow, but on the inside, it is white. The inside of the banana is yellow,', 'On the outside, the color of x is yellow. On the inside, the color of bananas is yellow. On the outside, the color']
the size of output is ************************************************torch.Size([4, 8, 1536])


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


['On the outside, the color of x is the same as the color of the bananas. On the inside, the color of the bananas is', 'On the outside, the color of x is the same as the color of the inside. The inside is a shade of green that is ', 'On the outside, the color of x is yellow, but on the inside, it is white. The inside of the banana is white,', 'On the outside, the color of x is yellow. On the inside, the color of bananas is yellow. On the outside, the color']
the size of output is ************************************************torch.Size([4, 8, 1536])


In [31]:
tokenizer([target_prompt for i in range(batch_size)],padding = True)

{'input_ids': [[1925, 279, 4889, 11, 279, 1894, 315, 856], [1925, 279, 4889, 11, 279, 1894, 315, 856], [1925, 279, 4889, 11, 279, 1894, 315, 856], [1925, 279, 4889, 11, 279, 1894, 315, 856]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [32]:
# Example batch of input IDs
input_ids_list = [
    [101, 7592, 102],          # "Hello"
    [101, 7632, 2045, 102],    # "Hi there"
    [101, 1045, 1005, 2310, 102]  # "I’ve"
]

pp = [{'input_ids' : ids_list,'abc':(0,1,22)}  for ids_list in input_ids_list]

In [33]:
pp[0 : 2]

[{'input_ids': [101, 7592, 102], 'abc': (0, 1, 22)},
 {'input_ids': [101, 7632, 2045, 102], 'abc': (0, 1, 22)}]

In [34]:
cc = tokenizer.tokenize(' apples')
print(cc)

['Ġapples']


In [35]:
dd = tokenizer.tokenize(' apples')
print(dd)

['Ġapples']


The biggest issue here is that 'Bananas' and 'bananas' are tookenized in a different way
And ' bananas' and 'bananas' are tokenized differently 

So I change my code a little bit so that we only sample those lines which has the strict match with subject

In [36]:
ee = tokenizer.convert_tokens_to_string(cc).lower()
print(ee)

 apples


In [37]:
ff = tokenizer.convert_tokens_to_string(dd).lower()
print(ff)

 apples


In [38]:
ee == ff

True

In [39]:
! nvidia-smi

Wed Mar  5 18:23:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off |   00000000:3B:00.0 Off |                  N/A |
| 25%   40C    P5             15W /  250W |    6278MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
